In [72]:
import pickle
import collections as col
import numpy as np
import random
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import math
import pdb
from sklearn import metrics
import time
import os
import PIL

import torch
import torch.nn as nn
from sklearn.metrics import roc_auc_score
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
from torchvision.models import resnet18, resnet34
import torch.nn.functional as F
from pytorch_datasets import DiagnosticInpainted
import models
import layers
import utilities.reading_images as reading_images
from utilities.loading import get_single_image
from torchvision.utils import save_image

%matplotlib inline

In [ ]:
class Pixel_CNN(nn.Module):
    def __init__(self, latent_dim):
        super(Pixel_CNN, self).__init__()
        
        # Conv2d: (input_channels, output_channels, kernel_size, padding)
        
        self.relu = nn.ReLU()
        
        self.model = nn.Sequential(
            nn.Conv2d(256, 256, (1, 1)),
            nn.ReLU(),
            nn.ConstantPad2d((1, 1, 0, 0), 0),
            nn.Conv2d(256, 256, (1, 3)),
            nn.ConstantPad2d((0, 0, 0, 1), 0),
            nn.Conv2d(256, 256, (2, 1)),
            nn.ReLU(),
            nn.Conv2d(256, 256, (1, 1))
        )

    def forward(self, latents):
        
        # latents: [B, C, H, W]
        cres = latents
        
        for _ in range(5):
            c = self.model(cres)
            cres = cres + c
        cres = self.relu(cres)
        return cres      

In [ ]:
# Yeah their implementation is wack. Not clear. 
# I'm going to go with WF. 

#

In [305]:
def raster_patchify(img, size = 80, overlap = 32):
    '''
    Left-to-right, top to bottom.
    Assumes img is (3, 240, 240).
    '''
    patches = []
     
    h = -32
    w = -32
    for i in range(6):
        h = h + 32
        for j in range(6):
            w = w + 32
            patches.append(img[:, h:h+size, w:w+size])
        w = -32
            
    return patches
    

In [306]:
patches = raster_patchify(trainset[0][0])
len(patches)

36

In [315]:
# plt.imshow(patches[35].permute(1, 2, 0))

# plt.imshow(trainset[0][0].permute(1, 2, 0))
# plt.scatter(80,80,color='r')
# plt.scatter(80+32,80,color='r')
# plt.scatter(80+64,80,color='r')
# plt.scatter(80+96,80,color='r')
# plt.scatter(80+128,80,color='r')
# plt.scatter(80+160,80,color='r')

In [322]:
def collate_fn(img_list):
    patches = []
    for (img, label) in img_list:
        img_patches = raster_patchify(img)
        patches.extend(img_patches)
        
    print('boom')

In [323]:
data_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(240),
    transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

trainset = datasets.ImageFolder(
    root = '/gpfs/data/geraslab/Vish/imagewoof320/imagewoof2-320/train/',
    transform = data_transform
)

train_dl = DataLoader(trainset, batch_size=10, shuffle=True, collate_fn=collate_fn)

valset = datasets.ImageFolder(
    root = '/gpfs/data/geraslab/Vish/imagewoof320/imagewoof2-320/val/',
    transform = data_transform
)

val_dl = DataLoader(valset, batch_size=10, shuffle=False)

In [324]:
for i, x in enumerate(train_dl):
    print('boom')

> <ipython-input-322-4b97eeea6591>(5)collate_fn()
-> img_patches = raster_patchify(img)
(Pdb) img
tensor([[[0.5647, 0.4745, 0.3882,  ..., 0.6392, 0.7020, 0.7647],
         [0.5490, 0.4353, 0.4431,  ..., 0.6353, 0.7373, 0.7725],
         [0.4078, 0.4510, 0.5529,  ..., 0.6471, 0.7373, 0.7647],
         ...,
         [0.4627, 0.4235, 0.4510,  ..., 0.6431, 0.5843, 0.5843],
         [0.5608, 0.4314, 0.3451,  ..., 0.6549, 0.6510, 0.6118],
         [0.5529, 0.4784, 0.4784,  ..., 0.6314, 0.6392, 0.6549]],

        [[0.5608, 0.4627, 0.3804,  ..., 0.6353, 0.7020, 0.7608],
         [0.5412, 0.4196, 0.4314,  ..., 0.6353, 0.7373, 0.7725],
         [0.4000, 0.4314, 0.5373,  ..., 0.6471, 0.7373, 0.7647],
         ...,
         [0.4471, 0.4314, 0.4667,  ..., 0.6314, 0.5725, 0.5725],
         [0.5529, 0.4471, 0.3647,  ..., 0.6431, 0.6392, 0.6000],
         [0.5569, 0.5020, 0.5020,  ..., 0.6196, 0.6275, 0.6431]],

        [[0.5529, 0.4824, 0.3961,  ..., 0.6353, 0.7098, 0.7725],
         [0.5373, 0.4392,

BdbQuit: 

In [157]:
len(trainset)

9025

In [160]:
trainset[50][0].shape

torch.Size([3, 240, 240])

In [109]:
class ResNet(nn.Module):
    """
    Adapted from torchvision ResNet, converted to v2
    """

    def __init__(self,
                 input_channels, num_filters,
                 first_layer_kernel_size, first_layer_conv_stride,
                 blocks_per_layer_list, block_strides_list, block_fn,
                 first_layer_padding=0,
                 first_pool_size=None, first_pool_stride=None, first_pool_padding=0,
                 growth_factor=2, norm_class="batch", num_groups=1):
        super(ResNet, self).__init__()
        self.first_conv = nn.Conv2d(
            in_channels=input_channels, out_channels=num_filters,
            kernel_size=first_layer_kernel_size,
            stride=first_layer_conv_stride,
            padding=first_layer_padding,
            bias=False,
        )
        # Diff: padding=SAME vs. padding=0
        self.first_pool = nn.MaxPool2d(
            kernel_size=first_pool_size,
            stride=first_pool_stride,
            padding=first_pool_padding,
        )
        self.norm_class = norm_class
        self.num_groups = num_groups

        block = self._resolve_block(block_fn)
        self.layer_list = nn.ModuleList()
        current_num_filters = num_filters
        self.inplanes = num_filters
        for i, (num_blocks, stride) in enumerate(zip(
                blocks_per_layer_list, block_strides_list)):
            self.layer_list.append(self._make_layer(
                block=block,
                planes=current_num_filters,
                blocks=num_blocks,
                stride=stride,
            ))
            current_num_filters *= growth_factor

        self.final_bn = layers.resolve_norm_layer(
            # current_num_filters // growth_factor
            current_num_filters // growth_factor * block.expansion,
            norm_class,
            num_groups
        )
        self.relu = nn.ReLU()
        self.initialize()

        # Expose attributes for downstream dimension computation
        self.num_filters = num_filters
        self.growth_factor = growth_factor
        self.block = block
        self.num_filter_last_seq = current_num_filters // growth_factor * block.expansion

    def forward(self, x, return_intermediate=False):
        intermediate = []
        h = self.first_conv(x)
        h = self.first_pool(h)

        if return_intermediate:
            intermediate.append(h)
        for i, layer in enumerate(self.layer_list):
            h = layer(h)
            if return_intermediate:
                intermediate.append(h)

        h = self.final_bn(h)
        h = self.relu(h)

        if return_intermediate:
            return h, intermediate
        else:
            return h

    @classmethod
    def _resolve_block(cls, block_fn):
        if block_fn == "normal":
            return layers.BasicBlockV2_dbt
        elif block_fn == "bottleneck":
            return layers.BottleneckV2_dbt
        else:
            raise KeyError(block_fn)

    def _make_layer(self, block, planes, blocks, stride=1):
        # downsample = None
        # if stride != 1 or self.inplanes != planes * block.expansion:
        downsample = nn.Sequential(
            nn.Conv2d(self.inplanes, planes * block.expansion,
                      kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(planes * block.expansion),
        )

        layers_ = [
            block(self.inplanes, planes, stride, downsample, self.norm_class, self.num_groups)
        ]
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers_.append(block(self.inplanes, planes, norm_class=self.norm_class, num_groups=self.num_groups))

        return nn.Sequential(*layers_)

    def initialize(self):
        for m in self.modules():
            self._layer_init(m)

    @classmethod
    def _layer_init(cls, m):
        if isinstance(m, nn.Conv2d):
            # From original
            nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        #             nn.init.xavier_normal_(m.weight)
        elif isinstance(m, nn.BatchNorm2d):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.GroupNorm):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)

    @classmethod
    def from_parameters(cls, parameters):
        return cls(
            input_channels=parameters["input_channels"],
            num_filters=parameters["num_filters"],
            first_layer_kernel_size=parameters["first_layer_kernel_size"],
            first_layer_conv_stride=parameters["first_layer_conv_stride"],
            first_layer_padding=parameters.get("first_layer_padding", 0),
            blocks_per_layer_list=parameters["blocks_per_layer_list"],
            block_strides_list=parameters["block_strides_list"],
            block_fn=parameters["block_fn"],
            first_pool_size=parameters["first_pool_size"],
            first_pool_stride=parameters["first_pool_stride"],
            first_pool_padding=parameters.get("first_pool_padding", 0),
            growth_factor=parameters.get("growth_factor", 2),
            norm_class=parameters.get("norm_class", "batch"),
            num_groups=parameters.get("num_groups", 1)
        )
    


In [144]:
class ResNet_22(nn.Module):

    def __init__(
            self,
            attention=False,
            dropout=0.0,
            hidden_size=256,

            # resnet hyperparameters
            #         input_channels=1,
            first_layer_kernel_size=7,
            first_layer_conv_stride=2,
            first_pool_size=3,
            first_pool_stride=2,
            first_layer_padding=0,
            first_pool_padding=0,
            growth_factor=2,

            # resnet22 settings
            num_filters=16,
            blocks_per_layer_list=[2, 2, 2, 2, 2],
            block_strides_list=[1, 2, 2, 2, 2],
            block_fn="normal",
            norm_class="group",
            num_groups=8,

            num_image_slices_per_net=1,
    ):
        super(ResNet_22, self).__init__()

        self.num_image_slices_per_net = num_image_slices_per_net

        self.dropout = nn.Dropout(p=dropout)
        self.relu = nn.ReLU()

        self.resnet = ResNet(
            input_channels=3,
            first_layer_kernel_size=first_layer_kernel_size,
            first_layer_conv_stride=first_layer_conv_stride,
            first_pool_size=first_pool_size,
            first_pool_stride=first_pool_stride,
            num_filters=num_filters,
            blocks_per_layer_list=blocks_per_layer_list,
            block_strides_list=block_strides_list,
            block_fn=block_fn,
            first_layer_padding=first_layer_padding,
            first_pool_padding=first_pool_padding,
            growth_factor=growth_factor,
            norm_class=norm_class,
            num_groups=num_groups,
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # use avgpool rather than torch.mean
        
    def forward(self, x):
        batch_size = x.shape[0]

        h = self.resnet(x)
        # Shape of pooled_h is [4, 256, 1, 1]
        pooled_h = self.avgpool(h)
        return pooled_h

In [162]:
class Identity(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x
    

model = ResNet_22()
model.eval()

boom = model(trainset[0][0].unsqueeze(0))

In [163]:
boom.shape

torch.Size([1, 256, 1, 1])